# eth_sendTransaction does not exist in Infura Test

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv
from web3 import Web3
from PIL import Image
from pinata import convert_data_to_json, pin_file_to_ipfs, pin_json_to_ipfs

load_dotenv()

w3 = Web3(Web3.HTTPProvider(os.getenv("goerli_infura_url")))

In [2]:
# Print if web3 is successfully connected
print(w3.isConnected())

# Get the latest block number
latest_block = w3.eth.block_number
print(latest_block)

# Get the balance of an account
address = '0x338B87AeBABc1EB4903257C8261726391C345a33'

balance = w3.eth.get_balance(address)
print(balance)

True
8063903
364420502326493623


In [3]:
def load_contract():

    # Load the contract ABI
    with open(Path('./contracts/compiled/nft_abi.json')) as f:
        contract_abi = json.load(f)

    # Set the contract address (this is the address of the deployed contract)
    contract_address = os.getenv("SMART_CONTRACT_ADDRESS")

    # Get the contract
    contract = w3.eth.contract(
        address=contract_address,
        abi=contract_abi
    )

    return contract


# Load the contract
contract = load_contract()

In [4]:
def pin_artwork(artwork_name, artwork_file):
    # Pin the file to IPFS with Pinata
    ipfs_file_hash = pin_file_to_ipfs(artwork_file.getvalue())

    # Build a token metadata file for the artwork
    token_json = {
        "name": artwork_name,
        "image": ipfs_file_hash
    }
    json_data = convert_data_to_json(token_json)

    # Pin the json to IPFS with Pinata
    json_ipfs_hash = pin_json_to_ipfs(json_data)

    return json_ipfs_hash, token_json


def pin_appraisal_report(report_content):
    json_report = convert_data_to_json(report_content)
    report_ipfs_hash = pin_json_to_ipfs(json_report)
    return report_ipfs_hash

In [5]:
import io
artwork_name = 'CharlotteBear'
artist_name = 'SC'
initial_appraisal_value = 100
# Convert It To Byte Array
img = Image.open("./Images/ybear.jpeg") 

byteIO = io.BytesIO()
img.save(byteIO, format='PNG')


In [6]:
# Use the `pin_artwork` helper function to pin the file to IPFS
artwork_ipfs_hash, token_json =  pin_artwork(artwork_name, byteIO)

artwork_uri = f"ipfs://{artwork_ipfs_hash}"

print(f"[Artwork IPFS Gateway Link](https://ipfs.io/ipfs/{artwork_ipfs_hash})")
print(f"[Artwork IPFS Image Link](https://ipfs.io/ipfs/{token_json['image']})")



{'IpfsHash': 'QmU9y4difZ9nxcmueaMsPxFYngkjt8wZMybxjYdB6qTUzK', 'PinSize': 45162, 'Timestamp': '2022-12-03T01:51:11.531Z'}
{'IpfsHash': 'bafkreiby4eueq62acfiu45vmcart4ankwoknamwmiy2io6sfdpn4fvphou', 'PinSize': 81, 'Timestamp': '2022-12-03T01:51:14.117Z'}
[Artwork IPFS Gateway Link](https://ipfs.io/ipfs/bafkreiby4eueq62acfiu45vmcart4ankwoknamwmiy2io6sfdpn4fvphou)
[Artwork IPFS Image Link](https://ipfs.io/ipfs/QmU9y4difZ9nxcmueaMsPxFYngkjt8wZMybxjYdB6qTUzK)


In [8]:
tokenURI = f"ipfs://{artwork_ipfs_hash}"
tokenJSON = token_json['image']

print(tokenURI)
print(tokenJSON)

ipfs://bafkreiby4eueq62acfiu45vmcart4ankwoknamwmiy2io6sfdpn4fvphou
QmU9y4difZ9nxcmueaMsPxFYngkjt8wZMybxjYdB6qTUzK


### Below failed example

In [29]:
tx_hash = contract.functions.registerArtwork(
        address,
        artwork_name,
        artist_name,
        int(initial_appraisal_value),
        artwork_uri,
        token_json['image']
    ).transact({'from': address, 'gas': 1000000})
receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Transaction receipt mined:")
print(dict(receipt))
print("You can view the pinned metadata file with the following IPFS Gateway Link")

{'IpfsHash': 'QmU9y4difZ9nxcmueaMsPxFYngkjt8wZMybxjYdB6qTUzK', 'PinSize': 45162, 'Timestamp': '2022-11-27T10:42:37.862Z', 'isDuplicate': True}
{'IpfsHash': 'bafkreib4clw2cy6add6qabzc2h6c3l6hbkeik577rxkzzhngaxuy4lfmlm', 'PinSize': 71, 'Timestamp': '2022-11-27T10:42:39.405Z', 'isDuplicate': True}


ValueError: {'code': -32601, 'message': 'The method eth_sendTransaction does not exist/is not available'}

In [2]:



account = w3.eth.coinbase
print(account)

# Get the information of a transaction
#tx = w3.eth.get_transaction('0x5c504ed432cb51138bcf09aa5e8a410dd4a1e204ef84bfed1be16dfba1b22060')
#print(tx)

HTTPError: 400 Client Error: Bad Request for url: https://eth-goerli.g.alchemy.com/v2/l7Gt4ILUu8dMQPHIixQHTqnxUPJyiAhz